# 4. Building a Chatbot

## Setup

In [10]:
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

assert os.environ["LANGSMITH_TRACING"] is not None
assert os.environ["LANGSMITH_API_KEY"] is not None
assert os.environ["LANGSMITH_PROJECT"] is not None
assert os.environ["OPENAI_API_KEY"] is not None

In [11]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

## 3.1 Introduction to LangGraph

### 3.1.1 LLMS are Stateless

- **By default, an LLM does not retain the context from previous invocations**. For example, if you tell an LLM your name in one invocation, it will not "remember" your name in the subsequent invocations

In [12]:
from langchain_core.messages import HumanMessage

intro_response = model.invoke([HumanMessage(content="Hi! I'm Bob")])
question_response = model.invoke([HumanMessage(content="What's my name?")])

In [13]:
intro_response.pretty_print()

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [14]:
question_response.pretty_print()

================================== Ai Message ==================================

I don't know your name unless you tell me. How can I assist you today?


- For the LLM to remember the name, the chat history has to be sent with each invocation

In [15]:
from langchain_core.messages import AIMessage

question_response_with_history = model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

question_response_with_history.pretty_print()

================================== Ai Message ==================================

Your name is Bob! How can I help you today, Bob?


### 3.1.2 LangGraph Introduction

- **LangGraph** is an open-source library from the creators of LangChain that is used to build A stateful, multi-step agent/workflow applications with reliable execution and persistence.
- Importantly, Human/Agent interactions are modeled as nodes in a graph which makes the orchestration between humans and agents visible and easy to debug.

#### Nodes
- A node is a single action: e.g., call an LLM, run a tool, query a database, or invoke custom code.
- Nodes can represent different actors (LLMs, tools, humans).
- In case of a human, the graph encodes when to pause for human input, how to resume, and how actors exchange state.

#### Edges
- Edges connect nodes and decide what runs next.
- They can be unconditional, conditional (branching), looping, or fan-out/fan-in (parallel paths and joins).
- Graph branches can run in parallel

#### Graph
- A workflow is the directed graph of nodes and edges.
- This makes the system explicit and debuggable: you can see the exact path the execution took.

#### State
- The workflow carries a state object (typically a structured dict/schema) that persists across nodes.
- Nodes read from state and propose updates to state rather than mutating it in place
- After each node runs, LangGraph materialises a new state version.
- This **immutability per step** yields reproducibility, diff-ability, and clear audit trails.

#### Checkpoints
- Each step can be checkpointed (state snapshot + metadata).
- Checkpoints and state can be stored in memory or external stores (DBs, object storage).

- With checkpoints, you can retry or reroute without losing prior work.
- This enables resume after failure/timeouts, deterministic replay, and precise debugging from any point.

#### Thread
- A thread is one concrete run of the graph (e.g., a single user session).
- Each thread has its own state history and checkpoints, isolating concurrent users/sessions cleanly.

When invoking the LLM, these training examples are placed before the actual question. The model uses them as guidance and then produces its final output